In [1]:
import os
import pandas as pd

In [2]:
# Getting the current work directory (cwd)
directories = ['ML4QS_Alex_120sec_On_Table',
               'ML4QS_Alex_120sec_Washing_Hands',
               'ML4QS_Alex_150sec_Eating',
               'ML4QS_Alex_150sec_Streching',
               'ML4QS_Alex_300sec_Walking',
               'ML4QS_Alex_300sec_Walking_Holding_Smartphone',
               'ML4QS_Alex_900sec_Running']

csvs = ('Accelerometer.csv',
        'Light.csv',
        'Location.csv',
        'Proximity.csv')

current_dir = os.getcwd()

files = []
for r, d, f in os.walk(current_dir):
    for file in f:
        if file.endswith(csvs):
            files.append(os.path.join(r, file))
                

for csv in csvs:
    frames = []
    for file in files:
        if file.endswith(csv):
#             pri
            time_path = os.path.join(os.path.dirname(file), 'meta', 'time.csv')
            time_frame = pd.read_csv(time_path)
            data_frame = pd.read_csv(file)
            start_time = float(time_frame.loc[time_frame['event'] == 'START', 'system time'])
            data_frame['Time (s)'] = ((data_frame.loc[:, 'Time (s)'] + start_time) * 10**9).astype('int64')
            
            if csv == 'Accelerometer.csv':
                 data_frame.rename({'Time (s)':'timestamp',
                               'Acceleration x (m/s^2)':'x',
                               'Acceleration y (m/s^2)':'y',
                               'Acceleration z (m/s^2)':'z'}, axis=1, inplace=True)
                    
            elif csv == 'Light.csv':
                data_frame.rename({'Time (s)':'timestamp',
                               'Illuminance (lx)':'illuminance'}, axis=1, inplace=True)
                
            elif csv == 'Location.csv':
                data_frame.rename({'Time (s)':'timestamp',
                               'Latitude (°)':'latitude',
                               'Longitude (°)':'longitude',
                               'Height (m)':'height',
                               'Velocity (m/s)':'velocity',
                               'Direction (°)':'direction',
                               'Horizontal Accuracy (m)':'horizontal_accuracy',
                               'Vertical Accuracy (m)':'vertical_accuracy'}, axis=1, inplace=True)
                
            elif csv == 'Proximity.csv':
                data_frame.rename({'Time (s)':'timestamp',
                               'Distance (cm)':'distance'}, axis=1, inplace=True)
            frames.append(data_frame)
    final_frame = pd.concat(frames, ignore_index = True)
    final_frame.to_csv('final/' + csv)

In [3]:
labels = pd.DataFrame(columns=['label', 'label_start', 'label_end'])
for directory in directories:
    path = os.path.join(current_dir, directory, 'meta', 'time.csv')
    frame = pd.read_csv(path)
    label_start = int(frame.loc[frame['event'] == 'START', 'system time'].item() * 10**9)
    label_end = int(frame.loc[frame['event'] == 'PAUSE', 'system time'].item() * 10**9)
    labels = labels.append({'label': directory, 'label_start':label_start, 'label_end':label_end}, ignore_index=True)
labels.to_csv('final/' + 'labels.csv')